In [19]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import dgl

node1 = pd.read_csv('node1.csv')
node2 = pd.read_csv('node2.csv')
node3 = pd.read_csv('node3.csv')
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def generate_words(row):
    fields = [
        'state', #
        'vapor_cloud_formation',#
        'combustible',#
        'confinement',#
        'mechanical_failure',#
        'release',#
        'overflow',#
        'vaporization',#
        'temperature',#
        'pressure',#
        'ignition',#
        'pre_mixture',
        'toxicity_of_sub',
        'dispersion'
    ]
    words = [f"{field} {row[field]}" for field in fields]
    return words

node1_word = node1.apply(generate_words, axis=1)
node2_word = node2.apply(generate_words, axis=1)
node3_word = node3.apply(generate_words, axis=1)

In [ ]:
node1_word_df = pd.DataFrame(node1_word.tolist(), columns=[
    'state',
    'vapor_cloud_formation',
    'combustible',
    'confinement',
    'mechanical_failure',
    'release',
    'overflow',
    'vaporization',
    'temperature',
    'pressure',
    'ignition',
    'pre_mixture',
    'toxicity_of_sub',
    'dispersion'
])
node2_word_df = pd.DataFrame(node2_word.tolist(), columns=[
    'state',
    'vapor_cloud_formation',
    'combustible',
    'confinement',
    'mechanical_failure',
    'release',
    'overflow',
    'vaporization',
    'temperature',
    'pressure',
    'ignition',
    'pre_mixture',
    'toxicity_of_sub',
    'dispersion'
])

node3_word_df = pd.DataFrame(node3_word.tolist(), columns=[
    'state',
    'vapor_cloud_formation',
    'combustible',
    'confinement',
    'mechanical_failure',
    'release',
    'overflow',
    'vaporization',
    'temperature',
    'pressure',
    'ignition',
    'pre_mixture',
    'toxicity_of_sub',
    'dispersion'
])
node2_word_df

,state,vapor_cloud_formation,combustible,confinement,mechanical_failure,release,overflow,vaporization,temperature,pressure,ignition,pre_mixture,toxicity_of_sub,dispersion
0,state liquid,vapor_cloud_formation True,combustible True,confinement high,mechanical_failure True,release continuous,overflow False,vaporization high,temperature high,pressure low,ignition delayed,pre_mixture True,toxicity_of_sub False,dispersion True
1,state liquid,vapor_cloud_formation True,combustible True,confinement false,mechanical_failure True,release continuous,overflow False,vaporization high,temperature low,pressure high,ignition instant,pre_mixture True,toxicity_of_sub False,dispersion True


In [22]:
node1_word_df['combined_sentence'] = node1_word_df.apply(lambda row: ','.join(row.astype(str)), axis=1)
node2_word_df['combined_sentence'] = node2_word_df.apply(lambda row: ','.join(row.astype(str)), axis=1)
node3_word_df['combined_sentence'] = node3_word_df.apply(lambda row: ','.join(row.astype(str)), axis=1)

In [23]:
node2_word_df['label'] = None
node2_word_df['scenario'] = None
node2_word_df['label'] = node2['label']

In [ ]:
import json

# load json file
absolute_path = "/home/chenyangtamu/graphdecoder/graphtoseq_3_nodes_no_equipment/data/dataset/graph_relations_labeled_structured.json"

with open(absolute_path, 'r') as file:
    scenarios_data = json.load(file)  

for i in range(len(node2_word_df)):
    label = node2_word_df["label"].iloc[i]  

    for scenario in scenarios_data: 
        if label.lower() == scenario["Hazard"].lower():
            
            pretty_text = " <newline> ".join(scenario["Scenarios"])
            node2_word_df.at[i, "scenario"] = pretty_text
            break

node2_word_df


,state,vapor_cloud_formation,combustible,confinement,mechanical_failure,release,overflow,vaporization,temperature,pressure,ignition,pre_mixture,toxicity_of_sub,dispersion,combined_sentence,label,scenario
0,state liquid,vapor_cloud_formation True,combustible True,confinement high,mechanical_failure True,release continuous,overflow False,vaporization high,temperature high,pressure low,ignition delayed,pre_mixture True,toxicity_of_sub False,dispersion True,"state liquid,vapor_cloud_formation True,combus...",VCE_liquid_continuous,High vaporization causes Vapor cloud formation...
1,state liquid,vapor_cloud_formation True,combustible True,confinement false,mechanical_failure True,release continuous,overflow False,vaporization high,temperature low,pressure high,ignition instant,pre_mixture True,toxicity_of_sub False,dispersion True,"state liquid,vapor_cloud_formation True,combus...",Normal,Normal causes Normal


In [26]:
node2_word_df.to_csv("node2_combined.csv", index=False, encoding="utf-8")

In [27]:
label_counts = node2_word_df["label"].value_counts()
print(label_counts)

label
VCE_liquid_continuous    1
Normal                   1
Name: count, dtype: int64


In [ ]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
from tqdm import tqdm
import pandas as pd

tokenizer = T5Tokenizer.from_pretrained('t5-small')
encoder = T5EncoderModel.from_pretrained('t5-small')
encoder.eval()  

def encode_series_to_embeddings(series_of_words, encoder, tokenizer, max_length=10, device=device):
    input_ids_all = []
    attention_masks_all = []

    print("Tokenizing...")

    for word_list in tqdm(series_of_words):
        encoding = tokenizer(
            word_list,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors='pt'
        )
        input_ids_all.append(encoding['input_ids'])        # [12, max_length]
        attention_masks_all.append(encoding['attention_mask'])

    input_ids = torch.stack(input_ids_all).to(device)              # [1600, 12, max_length]
    attention_mask = torch.stack(attention_masks_all).to(device)   # [1600, 12, max_length]

    batch_size, num_nodes, seq_len = input_ids.shape

    input_ids_flat = input_ids.view(-1, seq_len)             # [1600*12, max_length]
    attention_mask_flat = attention_mask.view(-1, seq_len)

    print("Encoding with T5...")

    with torch.no_grad():
        encoder = encoder.to(device)
        outputs = encoder(input_ids=input_ids_flat, attention_mask=attention_mask_flat)
        last_hidden_state = outputs.last_hidden_state 

    node_embeddings = last_hidden_state[:, 0, :]  

    node_embeddings = node_embeddings.view(batch_size, num_nodes, -1)

    return node_embeddings  # shape: [1600, 12, 512]


In [ ]:
import torch
from transformers import T5Tokenizer, T5EncoderModel
from tqdm import tqdm

def encode_series_to_embeddings(series_of_words, encoder, tokenizer, max_length, batch_size, device="cuda",):
    encoder = encoder.to(device)
    encoder.eval()

    all_embeddings = []

    print("Encoding in batches...")

    for i in tqdm(range(0, len(series_of_words), batch_size)):
        batch_graphs = series_of_words[i : i + batch_size]  # There are N graphs in each batch with 15 nodes

        input_ids_all = []
        attention_masks_all = []

        for word_list in batch_graphs:
            encoding = tokenizer(
                word_list,
                padding='max_length',
                truncation=True,
                max_length=max_length,
                return_tensors='pt'
            )
            input_ids_all.append(encoding['input_ids'])           # [15, max_length]
            attention_masks_all.append(encoding['attention_mask'])

        input_ids = torch.stack(input_ids_all).to(device)         # [B, 15, max_length]
        attention_mask = torch.stack(attention_masks_all).to(device)

        B, num_nodes, seq_len = input_ids.shape
        input_ids_flat = input_ids.view(-1, seq_len)              # [B*15, max_length]
        attention_mask_flat = attention_mask.view(-1, seq_len)

        with torch.no_grad():
            outputs = encoder(input_ids=input_ids_flat, attention_mask=attention_mask_flat)
            last_hidden_state = outputs.last_hidden_state         # [B*15, max_length, 512]

        node_embeddings = last_hidden_state[:, 0, :]              # [CLS] token → [B*15, 512]
        node_embeddings = node_embeddings.view(B, num_nodes, -1)  # reshape to [B, 15, 512]

        all_embeddings.append(node_embeddings.cpu())

    return torch.cat(all_embeddings, dim=0)  # fianl shape [N, 15, 512]


In [ ]:
node2_word_df['combined_sentence'][0]
token_lengths = [len(tokenizer.tokenize(text)) for text in node2_word_df['combined_sentence']]
print(f"最大长度: {max(token_lengths)}")
max_token_lengths = max(token_lengths)


最大长度: 72


In [ ]:
node1_sentence = node1_word_df['combined_sentence'].tolist()  # List[str]
node1_graph_texts = [[text] for text in node1_sentence]            # List[List[str]]
node2_sentence = node2_word_df['combined_sentence'].tolist()  # List[str]
node2_graph_texts = [[text] for text in node2_sentence]            # List[List[str]]
node3_sentence = node3_word_df['combined_sentence'].tolist()  # List[str]
node3_graph_texts = [[text] for text in node3_sentence]            # List[List[str]]

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
encoder = T5EncoderModel.from_pretrained('t5-small')

node1_embeddings = encode_series_to_embeddings(node1_graph_texts, encoder, tokenizer, max_length=max_token_lengths, device='cuda', batch_size=64)
node2_embeddings = encode_series_to_embeddings(node2_graph_texts, encoder, tokenizer, max_length=max_token_lengths, device='cuda', batch_size=64)
node3_embeddings = encode_series_to_embeddings(node3_graph_texts, encoder, tokenizer, max_length=max_token_lengths, device='cuda', batch_size=64)
print(node2_embeddings.shape)  # should be [7600, 1, 512]


Encoding in batches...


100%|██████████| 1/1 [00:00<00:00, 58.42it/s]


Encoding in batches...


100%|██████████| 1/1 [00:00<00:00, 99.93it/s]


Encoding in batches...


100%|██████████| 1/1 [00:00<00:00, 75.78it/s]

torch.Size([2, 1, 512])


In [33]:
combined = torch.cat([node1_embeddings, node2_embeddings, node3_embeddings], dim=1)
print(combined.shape)  # [7600, 1+X, 512]

torch.Size([2, 3, 512])


In [34]:
import dgl
import torch

def create_dgl_graph_from_embedding(embeddings: torch.Tensor):
    """
    Create a list of DGLGraphs from embeddings.
    Each graph has chain edges: node 0 -> node 1 -> ... -> node (n-1)

    The graphs will be created on the same device as embeddings.
    
    :param embeddings: Tensor of shape [batch_size, num_nodes, feature_dim]
    :return: List of DGLGraphs
    """
    batch_size, num_nodes, feature_dim = embeddings.shape
    device = embeddings.device

    graph_list = []

    for i in range(batch_size):
        g = dgl.DGLGraph().to(device)
        g.add_nodes(num_nodes)

        # Create chain edges: 0→1, 1→2, ..., (n-2)→(n-1)
        src_nodes = torch.arange(0, num_nodes - 1, device=device)
        dst_nodes = torch.arange(1, num_nodes, device=device)
        g.add_edges(src_nodes, dst_nodes)

        # Assign node features
        g.ndata['h'] = embeddings[i]

        graph_list.append(g)

    return graph_list


In [35]:
graph_list = create_dgl_graph_from_embedding(combined)

print(graph_list[0])
print(graph_list[0].ndata['h'].shape) 
dgl.save_graphs("graph_data.bin", graph_list)


Graph(num_nodes=3, num_edges=2,
      ndata_schemes={'h': Scheme(shape=(512,), dtype=torch.float32)}
      edata_schemes={})
torch.Size([3, 512])


/home/chenyangtamu/miniconda3/envs/graphdecoder310/lib/python3.10/site-packages/dgl/heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(
